# Cascade river basin extraction test: colab version

Prepared by Simon M Mudd, 20-Sept-2024

## Get lsdtopotools and lsdviztools

This installs lsdtopotools and lsdviztools in google colab.
`lsdtopotools` command line is installed from the university of edinburgh's debian server.

`lsdviztools` is installed via `pip`

This will take around 25 seconds.

In [ ]:
!wget https://pkgs.geos.ed.ac.uk/geos-jammy/pool/world/l/lsdtopotools2/lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null
!apt install ./lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null

In [ ]:
!pip install lsdviztools &> /dev/null

Check the versions.
This notebook was written with lsdviztools 0.4.12
and lsdtopotools version 0.9

In [ ]:
import lsdviztools
lsdviztools.__version__

In [ ]:
!lsdtt-basic-metrics -v

## Get gdal command line tools

We might need some gdal command line tools. This is used for raster manipulation. Installation will take maybe 10 seconds.

In [ ]:
!apt install gdal-bin &> /dev/null

!gdalinfo --version

## Mount your DEM from google drive

You will need to mount your DEM from google drive. Note that uploading the DEM directly into google colab (i.e., by dragging and dropping the file into the file system in a colab window) will take *forever*. So the recommended approach is to upload the DEM and driver file into a file folder in google drive and then mount it.

*Note: you will need to give google colab permission to do this on your drive.*

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

### Navigate to the directory where you keep the data.

You will need to modify the path below for your own data.
Note that the first three subdirectories on a google drive will always be `/content/drive/MyDrive/`

In [ ]:
%cd /content/drive/MyDrive/DEM_data/marggraf/

Lets just make sure this works by checking the working directory and the files:

You should see the DEM and your driver file.

In [ ]:
!pwd
!ls

### Deal with a USGS `.tar.gz` file

You might get your raster from opentopography, which delivers rasters in `.tar.gz` format. This is how you deal with that:

In [ ]:
!ls *.gz

In [ ]:
!tar -xzf rasters_USGS10m.tar.gz

In [ ]:
!ls *.tif

In [ ]:
!gdalwarp -t_srs '+proj=utm +zone=15 +north +datum=WGS84' -of ENVI -r cubic -tr 10 10 output_USGS10m.tif CR_USGS.bil

### Check your files for georeferencing

Your DEM *must* be in UTM using the WGS84 base CRS.
The EPSG code there *MUST* start with three digits that are either 326 or 327
If your EPSG code does not start with these digits *THE ANALYSIS WILL FAIL*

You can check the EPSG code like this (note you need to reflect your filename):

In [ ]:
!gdalsrsinfo -e CascadeRiver_DEM.bil

### Convert DEM using gdal if necessary

Use this command to convert a .tif file if necessary. You will need to change the filename, the UTM zone, and the resampling (`-tr`) to match the grid spacing of your raster.

In [ ]:
!gdalwarp -t_srs '+proj=utm +zone=15 +north +datum=WGS84' -of ENVI -r cubic -tr 10 10 CascadeRiver_DEM.tif CR.bil

## Just extract channels to see if there will be problems with the basin

In [ ]:
!lsdtt-basic-metrics channels_USGS.driver

Now plot the hillshade and the channels

In [ ]:
!ls

In [ ]:
### Plot points the hillshade
import lsdviztools.lsdmapwrappers as lsdmw
%matplotlib inline
Base_file = "CR_USGS"
DataDirectory = "./"
this_img = lsdmw.PrintPointsOverHillshade(DataDirectory,Base_file, column_for_plotting = "Stream Order",
                                          points_fname = "CR_USGS_CN.csv",
                                          scale_points = False, column_for_scaling = "Stream Order",
                                          scaled_data_in_log = True, max_point_size = 20, min_point_size = 2,
                                          cmap="Blues", save_fig=True, size_format="geomorphology")

In [ ]:
from IPython.display import display, Image
display(Image(filename=this_img, width=800))

## Now lets get a basin

We run the analysis with a call to `lsdtt-basic-metrics`

You will need to make sure the name of the driver file is correct for your example.

I have also placed the outlet file in my drive for this example.

In [ ]:
!ls

Just get the channels here. This will

In [ ]:
!lsdtt-basic-metrics basin_USGS.driver

## Look at the outputs

Now plot the basin

In [ ]:
#import lsdviztools.lsdmapwrappers as lsdmw
r_prefix = "CR_USGS"
Base_file = r_prefix

#%%capture
basins_img = lsdmw.PrintBasins_Complex(DataDirectory,Base_file,cmap="gist_earth",
                             size_format="geomorphology",dpi=400, save_fig = True,
                             out_fname_prefix = "basins_plot")

In [ ]:
print(basins_img)
from IPython.display import display, Image
display(Image(filename=basins_img, width=800))

In [ ]:
!ls *.csv

Lets see the columns in the chi data map

In [ ]:
import pandas as pd
df = pd.read_csv("CR_USGS_chi_data_map.csv")
df.columns

And plot the channel in the basin

In [ ]:
### Plot points the hillshade
import lsdviztools.lsdmapwrappers as lsdmw
%matplotlib inline
Base_file = "CR_USGS"
DataDirectory = "./"
this_img = lsdmw.PrintPointsOverHillshade(DataDirectory,Base_file, column_for_plotting = "chi",
                                          points_fname = "CR_USGS_chi_data_map.csv",
                                          scale_points = True, column_for_scaling = "drainage_area",
                                          scaled_data_in_log = True, max_point_size = 20, min_point_size = 2,
                                          cmap="Blues", save_fig=True, size_format="geomorphology")

In [ ]:
from IPython.display import display, Image
display(Image(filename=this_img, width=800))